In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
df=pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")

# Basic data manipulation

We're going to do a couple of things to the data from [covidtracking.com](https://covidtracking.com/).  

We're first going to reformat the dates to add hyphens between the year, month, and day, so `20200228` becomes `2020-02-28`.  We'll then `melt` the data frame so that each observation is in its own row, so that (for example) `state, date, positive, negative, hospitalized, icu` becomes `state, date, observation_type, observation_value`, where `observation_type` is one of `positive`, `negative`, `hospitalized`, or `icu`.

In [11]:
def datemunge(di):
    d = str(di)
    return "%s-%s-%s" % (d[0:4], d[4:6], d[6:8])

df["date"] = df["date"].apply(datemunge)

In [95]:
cleaned = pd.melt(df, 
                  id_vars=['date', 'state'], 
                  value_vars=list(set(df.columns) - set(['date', 'state'])), 
                  value_name="cases",
                  var_name="case type")

In [96]:
cleaned

,date,state,case type,cases
0,2020-04-09,AK,onVentilatorCurrently,NaN
1,2020-04-09,AL,onVentilatorCurrently,NaN
2,2020-04-09,AR,onVentilatorCurrently,31
3,2020-04-09,AS,onVentilatorCurrently,NaN
4,2020-04-09,AZ,onVentilatorCurrently,NaN
...,...,...,...,...
44707,2020-03-02,WA,deathIncrease,3
44708,2020-03-01,RI,deathIncrease,NaN
44709,2020-03-01,WA,deathIncrease,3
44710,2020-02-29,WA,deathIncrease,1


# Per-state results

In [110]:
def cases_for_state(state, show_points=False):
    case_types = ['death', 'positive', 'hospitalizedCumulative', 'hospitalizedCurrently', 'inIcuCurrently', 'inIcuCumulative']
    chart = alt.Chart(cleaned).\
                encode(alt.X("date:N"), 
                       alt.Y("cases", scale=alt.Scale(type="log")), 
                       alt.Color("case type", 
                                 sort=alt.EncodingSortField(field="cases", 
                                                            order="descending", 
                                                            op="max")),
                       tooltip=['date', 'state', 'case type', 'cases']).\
                transform_filter(alt.datum.state == state).\
                transform_filter(alt.datum.cases > 0).\
                transform_filter(alt.FieldOneOfPredicate("case type", case_types))
    
    return chart.mark_line() + chart.mark_point() if show_points else chart.mark_line()

cases_for_state("WY")


alt.Chart(...)

In [60]:
state = "WI"

base = alt.Chart(df).encode(alt.X("date:N")) 

alt.layer(
    base.mark_line(color="orange").encode(alt.Y("positive", scale=alt.Scale(type="log"))).transform_filter(alt.datum.state == state).transform_filter(alt.datum.positive > 0),
    base.mark_line(color="red").encode(alt.Y("hospitalizedCumulative", scale=alt.Scale(type="log"))).transform_filter(alt.datum.state == state).transform_filter(alt.datum.hospitalizedCumulative > 0),
    base.mark_line(color="darkred").encode(alt.Y("inIcuCumulative", scale=alt.Scale(type="log"))).transform_filter(alt.datum.state == state).transform_filter(alt.datum.hospitalizedCumulative > 0),
    base.mark_line(color="black").encode(alt.Y("death", scale=alt.Scale(type="log"))).transform_filter(alt.datum.state == state).transform_filter(alt.datum.death > 0)
)


alt.LayerChart(...)

In [24]:
chart = alt.Chart(df[df["positive"]>0]).mark_line().encode(alt.X("date:N"), 
                                 alt.Y("positive", scale=alt.Scale(type="log")),
                                 color=alt.Color("state", sort=alt.EncodingSortField(field="positive", order="descending", op="max")))

chart

alt.Chart(...)

In [98]:
domain = df["date"].sort_values(ascending=True).unique()

chart = alt.Chart(df).mark_line().encode(
    alt.X("date:N", scale=alt.Scale(domain=domain)), 
    color=alt.Color("state", sort=alt.EncodingSortField(field="positive", order="descending", op="max"))).transform_filter(alt.datum.positive > 0).resolve_axis(x="shared")
    

alt.vconcat(chart.encode(alt.Y("positive", scale=alt.Scale(type="log"))).transform_filter(alt.datum.positive > 0).resolve_scale(x="shared"), chart.encode(alt.Y("hospitalized", scale=alt.Scale(type="log"))).transform_filter(alt.datum.hospitalized > 0).resolve_scale(x="shared"))


alt.VConcatChart(...)